# CSE 622 Final Project
Spring 2023  
E Tracy

# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pygame as pg

pygame 2.3.0 (SDL 2.24.2, Python 3.11.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Setting up the ruleset

In [2]:
# Class for holding number values corresponding to distinct cell states
class States:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

## Conway's game of life

In [8]:
class Life:
    def __init__(self, screen, target_fps):
        self.screen = screen
        self.target_fps = target_fps
        self.clock = pg.time.Clock()
        self.w = self.screen.get_size()[0]
        self.h = self.screen.get_size()[1]
        self.states = States(alive=1, dead=0)
        self.bgc = (0, 0, 0)
        self.colors = [0]*len(self.states.__dict__)
        self.colors[self.states.alive] = (255, 127, 255)
        self.colors[self.states.dead] = (0, 0, 0)
        # used for randomly setting cells as alive at t=0
        self.probability_of_life = 0.18
        # may need to specify seed here
        self.rand = np.random.default_rng()
        self.generation = 0
        self.n_cells_changed = 0
        self.n_cells_alive = 0
        self.setup_room()
    
    def run(self):
        running = True
        while running:
            # handle events
            for event in pg.event.get():
                if event.type == pg.QUIT:
                    running = False

            # handle game logic here
            self.new_generation()
            self.plot_life()
            
            # screen locks
            while self.screen.get_locked():
                self.screen.unlock()
            # update the screen
            pg.display.flip()
            # limit the fps
            self.clock.tick(self.target_fps)
    
    def new_generation(self):
        # create a new generation using the ruleset
        self.generation += 1
        w,h = self.w,self.h
        # an array representing the number of neighbors (including self) of each cell in the room (not counting border rows or columns)
        neighbors = self.room[1:w + 1, 1:h + 1] \
            + self.room[1:w + 1, 0:h] \
            + self.room[1:w + 1, 2:h + 2] \
            + self.room[0:w, 1:h + 1] \
            + self.room[0:w, 0:h] \
            + self.room[0:w, 2:h + 2] \
            + self.room[2:w + 2, 1:h + 1] \
            + self.room[2:w + 2, 0:h] \
            + self.room[2:w + 2, 2:h + 2]
            
        # apply rules
        #   1. if cell is alive and
        #       a. has 2 alive neigbors, it survives
        #       b. has 3 alive neighbors, it surives
        #   2. if a dead cell has 3 neighbors, it becomes alive
        #   3. otherwise cell is dead or dies
        self.room[1:w+1, 1:h+1][neighbors[:,:] == 3] = self.states.alive # applying rule 1a and rule 2
        self.room[1:w+1, 1:h+1][(neighbors[:,:] < 3) | (neighbors[:,:] > 4)] = self.states.dead # applying rule 3 and implicitly 1b (no action taken)
        
        self.update_borders()
        
        # perform life counts
        check_frequency = 30 # every n generations
        if self.generation % check_frequency == 0:
            self.n_cells_changed = (self.check_room[1:w+1, 1:h+1] != self.room[1:w+1, 1:h+1]).sum()
            self.n_cells_alive = self.room[1:w+1, 1:h+1].sum()
            self.check_room = self.room.copy()
        
    def update_borders(self):
        w,h = self.w, self.h
        # handle wrapping edges around to other sides
        ## second to bottommost row is copied into top border row
        self.room[0:1, :] = self.room[w:w+1, :] # note, this weird indexing is so that the ndarray returned has shape (1, w+2) instead of shape (w+2,)
        ## second to topmost row is copied into bottom border row
        self.room[w+1:w+2, :] = self.room[1:2, :]
        ## second to rightmost column is copied into left border column
        self.room[:, 0:1] = self.room[:, h:h+1]
        ## second to leftmost column is copied into right border column
        self.room[:, h+1:h+2] = self.room[:, 1:2]
    
    def setup_room(self):
        # init the array to be the size of the screen plus a 1 pixel border for wrapping around (toroid topology)
        w,h = self.w, self.h
        self.room = self.rand.random(size=(w+2, h+2)) + self.probability_of_life
        self.room = self.room.astype(np.uint8)
        self.update_borders()
        # used for determining amount of cells that have changed in the current generation
        self.check_room = self.room.copy()
        self.gen = 0
        self.n_cells_changed = 0
        self.n_cells_alive = self.room[1:w+1, 1:h+1].sum()
    
    def plot_life(self):
        while self.screen.get_locked():
            self.screen.unlock()
        w,h = self.w, self.h
        rgbarr = pg.surfarray.pixels2d(self.screen)
        rgbarr[:,:] = self.room[1:w+1, 1:h+1] * self.get_24bit_color(self.colors[self.states.alive])
    
    def get_24bit_color(self, color : tuple):
        if len(color) != 3:
            return 0
        # given 8 bit color in rgb, 
        # 24bit_color = 256^2 r + 256 g + b 
        return color[0] * 256 ** 2 + color[1] * 256 + color[2]

pg.display.init()
disp_size = (800, 800)
screen = pg.display.set_mode(disp_size, pg.DOUBLEBUF)
pg.display.set_caption("Conways Game of Life")
life = Life(screen, 60)
life.run()
pg.quit()

# Social Dynamics

In [ ]:
# social dynamics ruleset
# cell states:
# 	dead
# 	neutral
# 	unhappy
# 	happy
	
# neighborhood = up,down,left,right,diagonals, aka 8-adjacent cells
# rules:
# 1.	if a cell is unhappy
# 	a. and has at least 1 happy neighbor, they become happy
# 	b. and has only unhappy neighbors, they stay unhappy
# 	c. and has no neighbors, they become neutral
# 2. if a cell is happy
# 	a. and has only unhappy neighbors, they become unhappy
# 	b. and has at least 1 happy neighbor, they stay happy
# 	c. and has no neighbors, they become neutral
# 3. if a cell is neutral
# 	a. and has no neighbors, they die
# 	b. and has majority unhappy neighbors, they become unhappy
# 	c. and has majority happy neighbors, they become happy
# 	d. and has equal happy and unhappy neighbors, they stay neutral
# 4. if a cell is dead
# 	a. and has 3 happy neighbors, they become happy
# 	b. and has 3 unhappy neighbors, they become unhappy
# 	c. and has 3 neutral neighbors, they become neutral

In [ ]:
class SocialDynamics:
    def __init__(self, screen, target_fps):
        self.screen = screen
        self.target_fps = target_fps
        self.clock = pg.time.Clock()
        self.w = self.screen.get_size()[0]
        self.h = self.screen.get_size()[1]
        self.states = States(
            dead = 0,
            neutral = 1,
            unhappy = 2,
            happy = 3
        )
        self.bgc = (0, 0, 0)
        self.colors = [0]*len(self.states.__dict__)
        self.colors[self.states.dead] = (0, 0, 0)
        self.colors[self.states.neutral] = (127, 127, 127)
        self.colors[self.states.unhappy] = (0, 0, 255)
        self.colors[self.states.happy] = (0, 255, 0)
        # used for randomly setting cell states at t=0
        self.p_neutral = 0.15
        self.p_happy = 0.15
        self.p_unhappy = 0.15
        # may need to specify seed here
        self.rand = np.random.default_rng()
        self.generation = 0
        self.n_cells_changed = 0
        self.n_cells_alive = 0
        
        self.setup_room()
    
    def run(self):
        running = True
        while running:
            # handle events
            for event in pg.event.get():
                if event.type == pg.QUIT:
                    running = False

            # handle game logic here
            self.new_generation()
            self.plot_life()
            
            # screen locks
            while self.screen.get_locked():
                self.screen.unlock()
            # update the screen
            pg.display.flip()
            # limit the fps
            self.clock.tick(self.target_fps)
    
    def new_generation(self):
        # create a new generation using the ruleset
        self.generation += 1
        w,h = self.w,self.h
        # an array representing the number of neighbors (including self) of each cell in the room (not counting border rows or columns)
        neighbors = self.room[1:w + 1, 1:h + 1] \
            + self.room[1:w + 1, 0:h] \
            + self.room[1:w + 1, 2:h + 2] \
            + self.room[0:w, 1:h + 1] \
            + self.room[0:w, 0:h] \
            + self.room[0:w, 2:h + 2] \
            + self.room[2:w + 2, 1:h + 1] \
            + self.room[2:w + 2, 0:h] \
            + self.room[2:w + 2, 2:h + 2]
            
        # apply rules
        #   1. if cell is alive and
        #       a. has 2 alive neigbors, it survives
        #       b. has 3 alive neighbors, it surives
        #   2. if a dead cell has 3 neighbors, it becomes alive
        #   3. otherwise cell is dead or dies
        self.room[1:w+1, 1:h+1][neighbors[:,:] == 3] = self.states.alive # applying rule 1a and rule 2
        self.room[1:w+1, 1:h+1][(neighbors[:,:] < 3) | (neighbors[:,:] > 4)] = self.states.dead # applying rule 3 and implicitly 1b (no action taken)
        
        self.update_borders()
        
        # perform life counts
        check_frequency = 30 # every n generations
        if self.generation % check_frequency == 0:
            self.n_cells_changed = (self.check_room[1:w+1, 1:h+1] != self.room[1:w+1, 1:h+1]).sum()
            self.n_cells_alive = self.room[1:w+1, 1:h+1].sum()
            self.check_room = self.room.copy()
        
    def update_borders(self):
        w,h = self.w, self.h
        # handle wrapping edges around to other sides
        ## second to bottommost row is copied into top border row
        self.room[0:1, :] = self.room[w:w+1, :] # note, this weird indexing is so that the ndarray returned has shape (1, w+2) instead of shape (w+2,)
        ## second to topmost row is copied into bottom border row
        self.room[w+1:w+2, :] = self.room[1:2, :]
        ## second to rightmost column is copied into left border column
        self.room[:, 0:1] = self.room[:, h:h+1]
        ## second to leftmost column is copied into right border column
        self.room[:, h+1:h+2] = self.room[:, 1:2]
    
    def setup_room(self):
        # init the array to be the size of the screen plus a 1 pixel border for wrapping around (toroid topology)
        w,h = self.w, self.h
        self.room = self.rand.random(size=(w+2, h+2)) + self.probability_of_life
        self.room = self.room.astype(np.uint8)
        self.update_borders()
        # used for determining amount of cells that have changed in the current generation
        self.check_room = self.room.copy()
        self.gen = 0
        self.n_cells_changed = 0
        self.n_cells_alive = self.room[1:w+1, 1:h+1].sum()
    
    def plot_life(self):
        while self.screen.get_locked():
            self.screen.unlock()
        w,h = self.w, self.h
        rgbarr = pg.surfarray.pixels2d(self.screen)
        rgbarr[:,:] = self.room[1:w+1, 1:h+1] * self.get_24bit_color(self.colors[self.states.alive])
    
    def get_24bit_color(self, color : tuple):
        if len(color) != 3:
            return 0
        # given 8 bit color in rgb, 
        # 24bit_color = 256^2 r + 256 g + b 
        return color[0] * 256 ** 2 + color[1] * 256 + color[2]

pg.display.init()
disp_size = (800, 800)
screen = pg.display.set_mode(disp_size, pg.DOUBLEBUF)
pg.display.set_caption("Social Dynamics")
sd = SocialDynamics(screen, 60)
sd.run()
pg.quit()

# Scratch Paper

In [55]:
rng = np.random.default_rng(seed=5)
w,h = 5,5
# room = rng.random((w+2, h+2)) + 0.3
room = np.eye(w+2, h+2)
room = room.astype(np.uint8)
print(room)
# print('shape', room.shape)
print()
# print(room[0:1, :], type(room[0:1, :]), ' shape:', room[0:1, :].shape)
# print(room[0, :], type(room[0, :]), ' shape:', room[0, :].shape)
room[0:1, :] = room[w:w+1, :]
room[w+1:w+2, :] = room[1:2, :]
room[:, 0:1] = room[:, h:h+1]
room[:, h+1:h+2] = room[:, 1:2]
print(room)

[[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]]

[[1 0 0 0 0 1 0]
 [0 1 0 0 0 0 1]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [1 0 0 0 0 1 0]
 [0 1 0 0 0 0 1]]


In [57]:
neighbors = room[1:w + 1, 1:h + 1] \
            + room[1:w + 1, 0:h] \
            + room[1:w + 1, 2:h + 2] \
            + room[0:w, 1:h + 1] \
            + room[0:w, 0:h] \
            + room[0:w, 2:h + 2] \
            + room[2:w + 2, 1:h + 1] \
            + room[2:w + 2, 0:h] \
            + room[2:w + 2, 2:h + 2]
print(neighbors)

[[3 2 1 1 2]
 [2 3 2 1 1]
 [1 2 3 2 1]
 [1 1 2 3 2]
 [2 1 1 2 3]]


# Setting up the room

In [8]:
# room is 800x800 with a 1 pixel wide kill boundary
room = np.zeros(shape=(802, 802), dtype=np.byte)

In [ ]:
def updateRoom(room, ruleset):
    for rule in ruleset:
        # apply the rule to each cell in the room
        pass
    
    return new_room

# pygame

In [6]:
pg.init()
pg.display.set_caption("social dynamics cellular automata")
# create a surface on screen of size
screen = pg.display.set_mode((800, 800))
# variable that controls the main loop
running = True
# main loop
while running:
    # event handling, gets all events from the event queue
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
    # game logic goes here
    pg.display.update()

pg.quit()